In [1]:
import numpy as np
import cProfile
import time

In [2]:
# ============================================================
#  Unvectorized ARX(2,3) simulator (loop over time, then tracks)
# ============================================================

def simulate_arx2_3_tracks_unvectorized(y_init, X, theta, sigma, eps_fn=None, rng=None):
    """
    Unvectorized ARX(2,3) simulation:
    loops over time, and for each time updates all tracks one by one.
    This matches the vectorized version's RNG-use order.
    """
    if rng is None:
        rng = np.random.default_rng()

    if eps_fn is None:
        def eps_fn(I_prev, rng):
            # IMPORTANT: vectorized so shape matches vectorized version
            return rng.normal(0.0, sigma, size=I_prev.shape)

    a1, a2, b1, b2, b3 = theta
    n_tracks, n_steps, _ = X.shape
    tracks = np.zeros((n_tracks, n_steps))

    # each track keeps its own state
    I_tm2 = y_init[:, 0].copy()
    I_tm1 = y_init[:, 1].copy()

    for t in range(n_steps):
        # generate innovations for all tracks at once
        eps = eps_fn(I_tm1, rng)      # shape (n_tracks,)
        for s in range(n_tracks):
            x1, x2, x3 = X[s, t]
            I_t = (a1 * I_tm1[s] +
                   a2 * I_tm2[s] +
                   b1 * x1 + b2 * x2 + b3 * x3 +
                   eps[s])
            tracks[s, t] = I_t
            I_tm2[s], I_tm1[s] = I_tm1[s], I_t

    return tracks

# ============================================================
#  Fully Vectorized ARX(2,3) simulator (loops over time only)
# ============================================================

def simulate_arx2_3_tracks_vectorized(y_init, X, theta, sigma, eps_fn=None, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    if eps_fn is None:
        def eps_fn(I_prev, rng):
            return rng.normal(0.0, sigma, size=I_prev.shape)

    a1, a2, b1, b2, b3 = theta
    n_tracks, n_steps, _ = X.shape
    tracks = np.zeros((n_tracks, n_steps))

    I_tm2 = y_init[:, 0].copy()
    I_tm1 = y_init[:, 1].copy()

    for t in range(n_steps):
        eps = eps_fn(I_tm1, rng)
        x1 = X[:, t, 0]
        x2 = X[:, t, 1]
        x3 = X[:, t, 2]

        I_t = (a1 * I_tm1 +
               a2 * I_tm2 +
               b1 * x1 + b2 * x2 + b3 * x3 +
               eps)

        tracks[:, t] = I_t
        I_tm2, I_tm1 = I_tm1, I_t

    return tracks

# ============================================================
#  Timing + Equality Test
# ============================================================

def timing_and_equality_test():
    theta = np.array([0.6, -0.2, 1.0, -0.3, 0.5])
    sigma = 2.0

    sizes = [
        (100, 50),
        (500, 50),
        (100, 100),
        (500, 100),
    ]

    results = []

    for n_tracks, n_steps in sizes:
        rng = np.random.default_rng(123)

        y_init = rng.normal(30, 5, size=(n_tracks, 2))
        X = rng.normal(size=(n_tracks, n_steps, 3))

        # ---- Unvectorized ----
        t0 = time.time()
        tracks1 = simulate_arx2_3_tracks_unvectorized(
            y_init, X, theta, sigma,
            eps_fn=None,
            rng=np.random.default_rng(999)
        )
        t_un = time.time() - t0

        # ---- Vectorized ----
        t0 = time.time()
        tracks2 = simulate_arx2_3_tracks_vectorized(
            y_init, X, theta, sigma,
            eps_fn=None,
            rng=np.random.default_rng(999)
        )
        t_vec = time.time() - t0

        same = np.allclose(tracks1, tracks2)

        results.append((n_tracks, n_steps, t_un, t_vec, same))

    return results

# Example usage (not executed here)
results = timing_and_equality_test()
for n_tracks, n_steps, t_un, t_vec, same in results:
    print(f"{n_tracks=} {n_steps=}: unvec={t_un:.5f}s vec={t_vec:.5f}s SAME={same}")

n_tracks=100 n_steps=50: unvec=0.00755s vec=0.00048s SAME=True
n_tracks=500 n_steps=50: unvec=0.03783s vec=0.00070s SAME=True
n_tracks=100 n_steps=100: unvec=0.01518s vec=0.00080s SAME=True
n_tracks=500 n_steps=100: unvec=0.07662s vec=0.00149s SAME=True


In [3]:
nS = 124#climInitLon.shape[0]
diS = da.from_array(np.arange(0,nS,1).astype(dtype=np.int32),chunks=(1,))

NameError: name 'da' is not defined

In [ ]:
bam(diS)

In [ ]:

seed = 42
rng = np.random.default_rng(seed=seed)

In [ ]:
N = 15
tmp = rng.uniform(0, 1, (4, N)).T
print(np.shape(tmp))
tmp

(15, 4)


array([[0.77395605, 0.22723872, 0.74476216, 0.80476436],
       [0.43887844, 0.55458479, 0.96750973, 0.38747838],
       [0.85859792, 0.06381726, 0.32582536, 0.2883281 ],
       [0.69736803, 0.82763117, 0.37045971, 0.6824955 ],
       [0.09417735, 0.6316644 , 0.46955581, 0.13975248],
       [0.97562235, 0.75808774, 0.18947136, 0.1999082 ],
       [0.7611397 , 0.35452597, 0.12992151, 0.00736227],
       [0.78606431, 0.97069802, 0.47570493, 0.78692438],
       [0.12811363, 0.89312112, 0.22690935, 0.66485086],
       [0.45038594, 0.7783835 , 0.66981399, 0.70516538],
       [0.37079802, 0.19463871, 0.43715192, 0.78072903],
       [0.92676499, 0.466721  , 0.8326782 , 0.45891578],
       [0.64386512, 0.04380377, 0.7002651 , 0.5687412 ],
       [0.82276161, 0.15428949, 0.31236664, 0.139797  ],
       [0.4434142 , 0.68304895, 0.8322598 , 0.11453007]])

In [ ]:
N = 15
tmp = [rng.uniform(0, 1) for iN in range(N)]
print(np.shape(tmp))
tmp

(15,)


[0.7739560485559633,
 0.4388784397520523,
 0.8585979199113825,
 0.6973680290593639,
 0.09417734788764953,
 0.9756223516367559,
 0.761139701990353,
 0.7860643052769538,
 0.12811363267554587,
 0.45038593789556713,
 0.37079802423258124,
 0.9267649888486018,
 0.6438651200806645,
 0.82276161327083,
 0.44341419882733113]

In [ ]:
np.shape(tmp)

(15, 4)

In [ ]:
def test(nday):

    dt = 1.0*60*60 #1 hr  # Time step in seconds (1 hour)
    #T = np.float64(nday)
    T = np.float64(15) #15-day period of the fourier series
    N = 15 #Number of sine waves
    nt = np.arange(0,nday*60*60*24, dt)
    F = np.zeros([nt.shape[0], 4])
    F1 = np.zeros([nt.shape[0], 4])
    #F = np.zeros([24,4])
    #F1 = np.zeros([24,4])

    ## this is 3? 4? nested for loops 
    for iff in range(0, 4, 1):
        #X = np.zeros([15]) ### i think this is not needed
        #X = [random.uniform(0,1) for iN in range(N)]

        ## switching from random.uniform to np.random.uniform, 
        ## which allows a size specification saves ~.1 seconds (when ndays = 1000)
        ## difference bewteen random and np.random seems to be whether
        ## and how the upper bound is included 
        X = np.random.uniform(0, 1, N)

        for itt in range(nt.shape[0]):
            # vectorizing for loops (1 second faster whan nday = 1000)
            iN = np.arange(1, N+1, 1)
            F[itt,iff] = (np.sqrt(2.0/np.sum(iN**(-3.0)))*\
                np.sum([(iN**(-3.0/2.0))*np.sin(2.0*np.pi*(iN*itt/(24.*T)+X[iN-1]))]))
            

In [ ]:
nday = 1000

dt = 1.0*60*60 #1 hr  # Time step in seconds (1 hour)
#T = np.float64(nday)
T = np.float64(15) #15-day period of the fourier series
N = 15 #Number of sine waves
nt = np.arange(0,nday*60*60*24, dt)
F = np.zeros([nt.shape[0], 4])
F1 = np.zeros([nt.shape[0], 4])
#F = np.zeros([24,4])
#F1 = np.zeros([24,4])

## this is 3? 4? nested for loops 
for iff in range(0, 4, 1):
    #X = np.zeros([15]) ### i think this is not needed
    #X = [random.uniform(0,1) for iN in range(N)]
    X = np.random.uniform(0, 1, N)

    for itt in range(nt.shape[0]):
        ## replace 
            # [iN**(-3.0) for iN in range(1,N+1,1)] with np.arange(1, N+1, 1)**(-3.0)
        F[itt, iff] = (np.sqrt(2.0)/np.sum(np.arange(1, N+1, 1)**(-3.0))*\
            np.sum([(iN**(-3.0/2.0))*np.sin(2.0*np.pi*(iN*itt/(24.*T)+X[iN-1]))
                for iN in range(1,N+1,1)])) ### figure out how to vectorize when self refrencing

        # F[itt,iff] = (np.sqrt(2.0/np.sum([iN**(-3.0) for iN in range(1,N+1,1)]))*\
        #     np.sum([(iN**(-3.0/2.0))*np.sin(2.0*np.pi*(iN*itt/(24.*T)+X[iN-1]))
        #         for iN in range(1,N+1,1)]))
        


In [ ]:
[(iN**(-3.0/2.0))*np.sin(2.0*np.pi*(iN*itt/(24.*T)+X[iN-1])) for iN in range(1,N+1,1)]

[np.float64(-0.998710993824007),
 np.float64(0.22121867595514488),
 np.float64(0.1883183079822308),
 np.float64(0.08505757367124661),
 np.float64(0.07765780302989167),
 np.float64(-0.06263209910580392),
 np.float64(-0.052826951179927736),
 np.float64(-0.04120469041084166),
 np.float64(-0.021889013311225666),
 np.float64(-0.025819413135683752),
 np.float64(0.027394132034963898),
 np.float64(0.02099534238828709),
 np.float64(0.021220985805777273),
 np.float64(-0.004059496492626352),
 np.float64(0.013739833488195198)]

In [ ]:
iN = np.arange(1, N+1, 1)
[(iN**(-3.0/2.0))*np.sin(2.0*np.pi*(iN*itt/(24.*T)+X[iN-1]))]

[array([-0.99871099,  0.22121868,  0.18831831,  0.08505757,  0.0776578 ,
        -0.0626321 , -0.05282695, -0.04120469, -0.02188901, -0.02581941,
         0.02739413,  0.02099534,  0.02122099, -0.0040595 ,  0.01373983])]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])